In [17]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-133680
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-133680


In [18]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

amlcompute_cluster_name= "cpu-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes= 4 )
aml_compute= ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
aml_compute.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [25]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import numpy as np
import train

# Specify parameter sampler
ps = RandomParameterSampling(main.args)
#{
  #   '--C' : np.logspace(-4, 4, 20),
  #  '--max_iter' : list(range(100,500,100))})
    

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor= 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = train.main()                                                          

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator= est,
                                    hyperparameter_sampling= ps, 
                                    policy= policy,
                                    primary_metric_metric= "Accuracy", 
                                    primary_metric_goma= PrimaryMetricGoal.MAXIMIZE, 
                                    max_total_runs= 20,
                                    max_concurrnr_runs=4)

NameError: name 'main' is not defined

In [20]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr= exp.submit(config=hdc)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output= True)
assert(hdr,get_status()=="completed")

<>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?


NameError: name 'hdc' is not defined

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run= hdr.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguements'])

model= best_run.register_model(model_name= 'best_model', model_path= 'outputs/model')

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds= TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv' )

In [26]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

TypeError: 'NoneType' object is not iterable

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= "classification",
    primary_metric= "Accuracy",
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [2]:
# Submit your automl run

remote_run= experiment.submit(automl_config, show_output= False)

In [ ]:
# Retrieve and save your best automl model.

import joblib
# Get your best run and save the model from that run.

best_run= remote_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguements'])

model= best_run.register_model(model_name= 'best_model', model_path= 'outputs/model')